In [ ]:
import numpy as np
import pandas as pd
import pickle
import h5py
from processing_helper_functions import final_processing
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from scipy.stats import sem

In [ ]:
# Variables to choose which downsampling technique, time series encoding technique, and other settings to use

# Which year of past data you would like to train on. One of: 0, 1, 2 where 0 is most recent and 2 is least recent
# If you want to train on more than one year, enter the least recent year to include
years_data = 0
# Set to True if you want to only train on 1 year of data
# Set to False to train on all data through years variable above
only_one = True
# Which time encoding technique should be used
# One of: 'all', 'slopes', 'all_ma', 'sma', 'ema'
encode_method = 'all'
# Which set of features should be used
# One of: 'baseline_demographics_cols', 'baseline_mci_cols', 'baseline_demographics_withmci_cols', 'baseline_mmse30_cols',
# 'baseline_linear_selected_features','simplified_cols_withapoe', 'simplified_cols_withoutapoe', 'all_cols'
feature_set = 'simplified_cols_withapoe'
# Which downsampling technique should be used
# One of: 'randdownsample', 'matcheddownsample', 'train', 'weighted'
sample_type = "train"

In [ ]:
encode_suffix = encode_method
if encode_method in ['all','current']:
    encode_suffix = str(years_data+1)
if encode_method == 'ema':
    encode_suffix = encode_method+str(half_life)
if only_one:
    save_suffix = '%s_%s_2yrprev_within3_singleyear%s'%(sample_type, feature_set, str(int(encode_suffix)-1))
else:
    save_suffix = '%s_%s_2yrprev_within3_yrsincluded%s'%(sample_type, feature_set, encode_suffix)

file_suffix = "2yrprev_within3"
with h5py.File("../DATA/PROCESSED/standardized_stacked_imputed/%s.h5"%file_suffix, 'r') as hf:
    ALL_SAMPLES = hf["samples"][:]
    ALL_FEATURES_TIME = hf["features"][:]
DATA = pd.read_csv("../DATA/PROCESSED/standardized/merged_kept_data_%s.csv"%file_suffix, index_col=0)
ALL_SAMPLES_df = pd.DataFrame(ALL_SAMPLES, columns=["projid","fu_year","onset_label_time","onset_label_time_binary"])

constructed_data = final_processing(encode_method, years_data, only_one, feature_set, DATA.columns[6:], ALL_FEATURES_TIME)

# Train Models

In [ ]:
# GBDT
accuracy_save = []
roc_auc_save = []
pr_auc_save = []
precision_save = []
recall_save = []
fpr_save = []
tpr_save = []

if sample_type in ['randdownsample','matcheddownsample']:
    train_samples_df = pd.read_csv("../DATA/PROCESSED/split_projids/%s_train_%s.txt"%(sample_type,file_suffix),index_col=0).drop_duplicates()
    train_samples_df["keep"]=1
    data_train = pd.merge(DATA, train_samples_df, on=['projid','fu_year'], how='left')
    train_idx = (data_train["keep"]==1).values
else:
    train_idx = ALL_SAMPLES_df["projid"].isin(np.loadtxt("../DATA/PROCESSED/split_projids/train_%s.txt"%(file_suffix))).values
valid_idx = ALL_SAMPLES_df["projid"].isin(np.loadtxt("../DATA/PROCESSED/split_projids/test_%s.txt"%(file_suffix))).values

## Code to test effect of dropping missing rows and highly missing cols
## To run: first scroll down to the "Code to run JBHI revision experiments" section 
##         and run the "Code to test effect of dropping missing rows and highly missing cols" cell.
##         Then uncomment the next 3 lines and run this cell.
#not_missing = ALL_SAMPLES_df.set_index(['projid','fu_year']).index.isin(pd.read_csv("../Data_Processing/JBHI_REV_IMPUTATION_EXPERIMENTS/to_keep_rows.csv").set_index(['projid','fu_year']).index)
#train_idx = np.logical_and(train_idx, not_missing)
#valid_idx = np.logical_and(valid_idx, not_missing)

## Code for between study (ROS vs MAP) validation
## To run: first scroll down to the "Code to run JBHI revision experiments" section 
##         and run the "Code for between study (ROS vs MAP) validation" cell.
##         Then uncomment the next 4 lines and run this cell.
#train_exp = np.intersect1d(np.loadtxt("../DATA/PROCESSED/split_projids/train_%s.txt"%(file_suffix)), MAP_ids)
#valid_exp = np.intersect1d(np.loadtxt("../DATA/PROCESSED/split_projids/test_%s.txt"%(file_suffix)), MAP_ids)
#train_idx = ALL_SAMPLES_df["projid"].isin(train_exp).values
#valid_idx = ALL_SAMPLES_df["projid"].isin(valid_exp).values
    

label_col = np.where(ALL_SAMPLES_df.columns.values=='onset_label_time_binary')[0][0]
label_tr = ALL_SAMPLES[train_idx][:, label_col]
label_val = ALL_SAMPLES[valid_idx][:, label_col] 
data_tr = constructed_data[train_idx]
data_val = constructed_data[valid_idx]

if sample_type == 'weighted':
    weight_tr = label_tr.astype(float)
    weight_tr[weight_tr==0] = np.sum(weight_tr)/(weight_tr.shape[0]-np.sum(weight_tr))
    dmat_train = xgb.DMatrix(data_tr.fillna(-999.0), label=label_tr, missing=-999.0, weight=weight_tr)
else:
    dmat_train = xgb.DMatrix(data_tr.fillna(-999.0), label=label_tr, missing=-999.0)
dmat_test = xgb.DMatrix(data_val.fillna(-999.0), label=label_val, missing=-999.0)
num_round = 50
param = {'silent':1, 'min_child_weight':0.25, 'eta':0.1, 'max_depth': 4, 'objective': 'binary:logistic', 'eval_metric': ['error','logloss']}
model = xgb.train(param, dmat_train, num_boost_round=num_round, evals=[(dmat_train,'train'), (dmat_test,'eval')], early_stopping_rounds=5, verbose_eval=False)

predr = model.predict(dmat_test, ntree_limit=model.best_iteration)
predr_df = ALL_SAMPLES_df[["projid","fu_year"]][valid_idx]
predr_df['predr'] = predr
if feature_set == 'baseline_mci_cols':
    predr = data_val['mci']
accuracy_save.append(metrics.accuracy_score(label_val,np.round(predr)))
fpr, tpr, _ = metrics.roc_curve(label_val,predr)
fpr_save = fpr
tpr_save = tpr
roc_auc_save.append(metrics.auc(fpr,tpr))
precision, recall, _ = metrics.precision_recall_curve(label_val,predr)
precision_save.append(precision)
recall_save.append(recall)
pr_auc_save.append(metrics.auc(recall, precision))
new_results = [accuracy_save,roc_auc_save,pr_auc_save,precision_save,recall_save,fpr_save,tpr_save]
#print(new_results[0:3])
#predr_df.to_csv('../results/xgb_preds_%s.csv'%(save_suffix))
#pickle.dump(new_results, open('../results/xgb_%s.p'%(save_suffix), 'wb'))
#pickle.dump(model, open('../results/xgb_%s.dat'%(save_suffix), "wb"))

In [ ]:
# Linear
accuracy_save = []
roc_auc_save = []
pr_auc_save = []
precision_save = []
recall_save = []
fpr_save = []
tpr_save = []

if sample_type in ['randdownsample','matcheddownsample']:
    train_samples_df = pd.read_csv("../DATA/PROCESSED/split_projids/%s_train_%s.txt"%(sample_type,file_suffix),index_col=0).drop_duplicates()
    train_samples_df["keep"]=1
    data_train = pd.merge(DATA, train_samples_df, on=['projid','fu_year'], how='left')
    train_idx = (data_train["keep"]==1).values
else:
    train_idx =ALL_SAMPLES_df["projid"].isin(np.loadtxt("../DATA/PROCESSED/split_projids/train_%s.txt"%(file_suffix))).values
valid_idx =ALL_SAMPLES_df["projid"].isin(np.loadtxt("../DATA/PROCESSED/split_projids/test_%s.txt"%(file_suffix))).values

label_col = np.where(ALL_SAMPLES_df.columns.values=='onset_label_time_binary')[0][0]
label_tr = ALL_SAMPLES[train_idx][:, label_col]
label_val =ALL_SAMPLES[valid_idx][:, label_col] 
data_tr = constructed_data[train_idx]
data_val = constructed_data[valid_idx]

if sample_type == 'weighted':
    weight_tr = label_tr.astype(float)
    weight_tr_val = np.sum(weight_tr)/(weight_tr.shape[0]-np.sum(weight_tr))
    clf = LogisticRegression(class_weight={0: weight_tr_val, 1: 1})
else:
    clf = LogisticRegression()
clf.fit(data_tr, label_tr)

predr = clf.predict_proba(data_val)[:,1]
predr_df = ALL_SAMPLES_df[["projid","fu_year"]][valid_idx]
predr_df['predr'] = predr
accuracy_save.append(clf.score(data_val, label_val))
roc_auc_save.append(metrics.roc_auc_score(label_val, clf.decision_function(data_val)))
pr_auc_save.append(metrics.average_precision_score(label_val, clf.decision_function(data_val)))   
fpr_save, tpr_save, _ = metrics.roc_curve(label_val, clf.decision_function(data_val))
precision_save, recall_save, _ = metrics.precision_recall_curve(label_val, clf.decision_function(data_val))

new_results = [accuracy_save,roc_auc_save,pr_auc_save,precision_save,recall_save,fpr_save,tpr_save]
#print(new_results[0:3])
#predr_df.to_csv('../results/xgb_preds_%s.csv'%(save_suffix))
#pickle.dump(new_results, open('../results/linear_%s.p'%(save_suffix), 'wb'))
#pickle.dump(clf, open('../results/linear_%s.dat'%(save_suffix), "wb"))

# Code to run JBHI revision experiments

In [ ]:
## Code to test effect of dropping missing rows and highly missing cols
## To run: first run this cell. Then scroll up to the Train XGB cell and 
##         uncomment the 3 lines underneath the "Code to test effect of dropping missing rows and highly missing cols" comment
todrop_cols = np.loadtxt("../Data_Processing/JBHI_REV_IMPUTATION_EXPERIMENTS/to_drop_cols.txt", dtype=str)
for col in todrop_cols:
    constructed_data.drop(col, axis=1, inplace=True)

In [ ]:
## Code for between study (ROS vs MAP) validation
## To run: first run this cell. Then scroll up to the Train XGB cell and 
##         uncomment the 4 lines underneath the "Code for between study (ROS vs MAP) validation" comment
DATA_dope = pd.read_csv("../DATA_dope/PROCESSED/standardized/merged_kept_data_new_%s.csv"%file_suffix, index_col=0)
ROSMAP_ids = DATA_dope[['projid','study']].drop_duplicates()
ROS_ids = ROSMAP_ids['projid'].loc[ROSMAP_ids['study'] == 'ROS ']
MAP_ids = ROSMAP_ids['projid'].loc[ROSMAP_ids['study'] == 'MAP ']

In [ ]:
## Bootstrap standard error intervals for accuracy, AUROC, and AUPRC metrics
n_bootstraps = 1000
rng_seed = len(label_val)  # control reproducibility
bootstrapped_accuracy_scores = []
bootstrapped_auroc_scores = []
bootstrapped_auprc_scores = []

rng = np.random.RandomState(rng_seed)
for i in range(n_bootstraps):
    # bootstrap by sampling with replacement on the prediction indices
    indices = rng.randint(0, len(predr), len(predr))

    bootstrapped_accuracy_scores.append(metrics.accuracy_score(label_val[indices], np.round(predr[indices])))
    fpr, tpr, _ = metrics.roc_curve(label_val[indices],predr[indices])
    bootstrapped_auroc_scores.append(metrics.auc(fpr,tpr))
    precision, recall, _ = metrics.precision_recall_curve(label_val[indices],predr[indices])
    bootstrapped_auprc_scores.append(metrics.auc(recall, precision))
print(sem(bootstrapped_accuracy_scores))
print(sem(bootstrapped_auroc_scores))
print(sem(bootstrapped_auprc_scores))

In [ ]:
## Calculate confusion matrix
matrix = metrics.confusion_matrix(label_val.astype(bool), np.where(predr > 0.5, 1, 0), labels=[0, 1])
tn, fp, fn, tp = matrix.ravel()
sensitivity = tp/(tp+fn)
specificity = tn/(tn+fp)
print(matrix)
print(sensitivity)
print(specificity)

In [ ]:
## Calculate IDI
## Must have new_val calculated using correct model using predr previously in order to run
#new_val = np.mean(predr[label_val==1.0])-np.mean(predr[label_val==0.0])
old_val = np.mean(predr[label_val==1.0])-np.mean(predr[label_val==0.0])
print(new_val - old_val)
print((new_val - old_val)/old_val)